In [2]:
!pip install -U spacy==2.3.5

In [3]:
import spacy
print(spacy.__version__)

2.3.5


In [4]:
# Had to run this to make something below work.
# This is a donwgrade of numpy to a version without a bug.
# This is a temp solution until a new version of numpy has solved the bug.
!pip install numpy==1.19.3 --user

In [5]:
def is_float(n):
    try:
        support_float_with_norwegian_format = n.replace(',','.')
        float_n = float(support_float_with_norwegian_format)
    except ValueError:
        return False
    else:
        return True
    
# test
print(is_float('test'))
print(is_float('4'))
print(is_float('4.5'))
print(is_float('4,5'))

False
True
True
True


In [6]:
def is_int(n):
    try:
        float_n = float(n)
        int_n = int(float_n)
    except ValueError:
        return False
    else:
        return float_n == int_n

# test
print(is_int('test'))
print(is_int('4'))
print(is_int('4.5'))

False
True
False


In [7]:
text = "overstyrmann på skip med bruttotonnasje opp til 3000 i ubegrenset fartsområde. skip med bruttotonnasje over 500. Slike fartøy i innenriksfart uansett byggedato med bruttotonnasje 50 eller mer."

In [10]:
import spacy
from spacy.lang.en import English
from spacy.matcher import Matcher

# Blank model.
# Does not have a POS tagger.
nlp = English()

doc = nlp(text)

matcher = Matcher(nlp.vocab)

#
# START - spaCy patterns
#

# GROSS_TONNAGE
patterns = [
    [
        {"LOWER": {"IN": ["bruttotonnasje"]}},
        {"LOWER": {"IN": ["opp", "ned"]}},
        {"LOWER": {"IN": ["til"]}},
        {'IS_DIGIT': True}
    ],
    [
        {"LOWER": {"IN": ["bruttotonnasje"]}},
        {"LOWER": {"IN": ["under", "over"]}},
        {'IS_DIGIT': True}
    ],
    [
        {"LOWER": {"IN": ["bruttotonnasje"]}},
        {'IS_DIGIT': True},
        {"LOWER": {"IN": ["eller"]}},
        {"LOWER": {"IN": ["mer", "mindre"]}}
    ]
]

matcher.add("GROSS_TONNAGE", patterns)

#
# END - spaCy patterns
#

result = []

for match_id, token_start, token_end in matcher(doc):

    match_id_as_string = nlp.vocab.strings[match_id]
    final_token_start = token_start
    final_token_end = token_end

    spacy_pattern_detection = doc[token_start:token_end]
    spacy_pattern_detection_as_lower_text = spacy_pattern_detection.text.lower()
    
    #
    # convert token_span to char_span.
    # char_span is needed to display correctly withdisplacy.render().
    #
    span = doc[final_token_start:final_token_end]
    span_char_start = span[0].idx
    span_char_end = span[-1].idx + len(span[-1].text)

    # return result
    identified_entity = {'start': span_char_start, 'end': span_char_end, 'label': match_id_as_string}
    result.append(identified_entity)

# display result with spacy styling
display_this_result = {'text': text, 'ents': sorted(result, key=lambda x: x['start']), 'title': 0}
options = {"colors": {"GROSS_TONNAGE": "darksalmon"}}
from spacy import displacy
displacy.render(display_this_result, style='ent', manual=True, options=options)

# Functions run from .py file

In [9]:
#
# Something that must be done 
# to make the notebook reload a function
# from a costum .py file.
#

%reload_ext autoreload
%autoreload 2

import importlib

import spacy_matching_rule_identify_GROSS_TONNAGE_no

from spacy_matching_rule_identify_GROSS_TONNAGE_no import identify_GROSS_TONNAGE_in_text

importlib.reload(spacy_matching_rule_identify_GROSS_TONNAGE_no)

################
################

identify_GROSS_TONNAGE_in_text(text)

[{'start': 25, 'end': 52, 'label': 'GROSS_TONNAGE'},
 {'start': 88, 'end': 111, 'label': 'GROSS_TONNAGE'}]